In this example, we will use tensorflow v1 (version 1.15) to create a simple MLP model, and transfer the application to Cluster Serving step by step.

This tutorial is recommended for Tensorflow v1 user only. If you are not Tensorflow v1 user, the keras tutorial [here](#keras-to-cluster-serving-example.ipynb) is more recommended.

### Original Tensorflow v1 Application

In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

We first define the Tensorflow graph, and create some data.

In [2]:
g = tf.Graph()
with g.as_default():
   
    # Graph Inputs
    features = tf.placeholder(dtype=tf.float32, 
                              shape=[None, 2], name='features')
    targets = tf.placeholder(dtype=tf.float32, 
                             shape=[None, 1], name='targets')

    # Model Parameters
    weights = tf.Variable(tf.zeros(shape=[2, 1], 
                          dtype=tf.float32), name='weights')
    bias = tf.Variable([[0.]], dtype=tf.float32, name='bias')
    

    
    # Forward Pass
    linear = tf.add(tf.matmul(features, weights), bias, name='linear')
    ones = tf.ones(shape=tf.shape(linear)) 
    zeros = tf.zeros(shape=tf.shape(linear))
    prediction = tf.where(condition=tf.less(linear, 0.),
                          x=zeros, 
                          y=ones, 
                          name='prediction')
    
    # Backward Pass
    errors = targets - prediction
    weight_update = tf.assign_add(weights, 
                                  tf.reshape(errors * features, (2, 1)),
                                  name='weight_update')
    bias_update = tf.assign_add(bias, errors,
                                name='bias_update')
    
    train = tf.group(weight_update, bias_update, name='train')
    
    saver = tf.train.Saver(name='saver')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
import numpy as np
x_train, y_train = np.array([[1,2],[3,4],[1,3]]), np.array([1,2,1])
x_train.shape, y_train.shape

((3, 2), (3,))

### Export TensorFlow SavedModel
Then, we train the graph and in the `with tf.Session`, we save the graph to SavedModel. The detailed code is following, and we could see the prediction result is `[1]` with input `[1,2]`.

In [5]:
with tf.Session(graph=g) as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(5):
        for example, target in zip(x_train, y_train):
            feed_dict = {'features:0': example.reshape(-1, 2),
                         'targets:0': target.reshape(-1, 1)}
            _ = sess.run(['train'], feed_dict=feed_dict)


    w, b = sess.run(['weights:0', 'bias:0'])    
    print('Model parameters:\n')
    print('Weights:\n', w)
    print('Bias:', b)

    saver.save(sess, save_path='perceptron')
    
    pred = sess.run('prediction:0', feed_dict={features: x_train})
    print(pred)
    
    # in this session, save the model to savedModel format
    inputs = dict([(features.name, features)])
    outputs = dict([(prediction.name, prediction)])
    inputs, outputs
    tf.saved_model.simple_save(sess, "/tmp/mlp_tf1", inputs, outputs)

Model parameters:

Weights:
 [[15.]
 [20.]]
Bias: [[5.]]
[[1.]
 [1.]
 [1.]]
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/mlp_tf1/saved_model.pb


### Deploy Cluster Serving
After model prepared, we start to deploy it on Cluster Serving.

First install Cluster Serving

In [6]:
! pip install bigdl-serving

In [7]:
import os
! mkdir cluster-serving
os.chdir('cluster-serving')
! cluster-serving-init

Trying to find config file in  /home/user/anaconda3/envs/tf1/lib/python3.7/site-packages/bigdl/conf/config.yaml
Config file found in pip package, copying...
Config file ready.
Cluster Serving has been properly set up.
You did not specify ANALYTICS_ZOO_VERSION, will download 0.9.0
ANALYTICS_ZOO_VERSION is 0.9.0
BIGDL_VERSION is 0.12.1
SPARK_VERSION is 2.4.3
2.4
You are installing Cluster Serving by pip, downloading...


In [8]:
! tail wget-log

  2800K .......... .......... .......... .......... ..........  0% 11.8M 19m20s
  2850K .......... .......... .......... .......... ..........  0% 11.3M 19m1s
  2900K .......... .......... .......... .......... ..........  0% 8.60M 18m43s
  2950K .......... .......... .......... .......... ..........  0% 11.9M 18m25s
  3000K .......... .......... .......... .......... ..........  0% 11.8M 18m7s
  3050K .......... .......... .......... .......... ..........  0%  674K 18m4s
  3100K .......... .......... .......... .......... ..........  0%  418K 18m9s
  3150K .......... .......... .......... .......... ..........  0% 1.05M 18m0s
  3200K .......... .......... .......... .......... ..........  0%  750K 17m56s
  3250K .......... .......... .......... ....

In [ ]:
# if you encounter slow download issue like above, you can just use following command to download
# ! wget https://repo1.maven.org/maven2/com/intel/analytics/bigdl/bigdl-bigdl_0.12.1-spark_2.4.3/0.9.0/bigdl-bigdl_0.12.1-spark_2.4.3-0.9.0-serving.jar

# if you are using wget to download, or get "bigdl-xxx-serving.jar" after "ls", please call mv *serving.jar bigdl.jar after downloaded.

In [9]:
# After initialization finished, check the directory
! ls

bigdl-bigdl_0.12.1-spark_2.4.3-0.9.0-serving.jar  config.yaml  wget-log


In [10]:
# Call mv *serving.jar bigdl.jar as mentioned above
! mv *serving.jar bigdl.jar

In [11]:
! ls

config.yaml  wget-log  bigdl.jar


We config the model path in `config.yaml` to following (the detail of config is at [Cluster Serving Configuration](https://github.com/intel-analytics/bigdl/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#2-configuration))

In [ ]:
## BigDL Cluster Serving

model:
  # model path must be provided
  path: /tmp/mlp_tf1

In [13]:
! head config.yaml

## BigDL Cluster Serving

model:
  # model path must be provided
  path: /tmp/mlp_tf1
  # name, default is serving_stream, you need to specify if running multiple servings
  name:
data:
  # default, localhost:6379
  src:


### Start Cluster Serving

Cluster Serving requires Flink and Redis installed, and corresponded environment variables set, check [Cluster Serving Installation Guide](https://github.com/intel-analytics/bigdl/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#1-installation) for detail.

Flink cluster should start before Cluster Serving starts, if Flink cluster is not started, call following to start a local Flink cluster.

In [14]:
! $FLINK_HOME/bin/start-cluster.sh

Starting cluster.
Starting standalonesession daemon on host user-PC.
Starting taskexecutor daemon on host user-PC.


After configuration, start Cluster Serving by `cluster-serving-start` (the detail is at [Cluster Serving Programming Guide](https://github.com/intel-analytics/bigdl/blob/master/docs/docs/ClusterServingGuide/ProgrammingGuide.md#3-launching-service))

In [18]:
! cluster-serving-start

model_path="/tmp/mlp_tf1"
redis_timeout="5000"
Redis maxmemory is not set, using default value 8G
redis server started, please check log in redis.log
OK
OK
OK
redis config maxmemory set to 8G
OK
OK
Starting new Cluster Serving job.
Cluster Serving job submitted, check log in log-cluster_serving-serving_stream.txt
To list Cluster Serving job status, use cluster-serving-cli list
{maxmem=null, timeout=5000}timeout getted: 5000


### Prediction using Cluster Serving
Next we start Cluster Serving code at python client.

In [22]:
from bigdl.serving.client import InputQueue, OutputQueue
input_queue = InputQueue()
# Use async api to put and get, you have pass a name arg and use the name to get
arr = np.array([1,2])
input_queue.enqueue('my-input', t=arr)
output_queue = OutputQueue()
prediction = output_queue.query('my-input')
# Use sync api to predict, this will block until the result is get or timeout
prediction = input_queue.predict(arr)

redis group exist, will not create new one
redis group exist, will not create new one
Write to Redis successful
redis group exist, will not create new one
Write to Redis successful


In [23]:
prediction

array([1.], dtype=float32)

The `prediction` result would be the same as using Tensorflow.

This is the end of this tutorial. If you have any question, you could raise an issue at [BigDL Github](https://github.com/intel-analytics/bigdl/issues).